In [143]:
import pandas as pd 
from decimal import Decimal, ROUND_HALF_EVEN
from datetime import *
%config Completer.use_jedi = False
from dateutil.relativedelta import relativedelta

# Backlog
* Criar evolução do patrimonio atual até a curva perpétua e curva de consumo.
* Criar a evolução do patrimonio para usufruto para a curva perpétua e de consumo
* Criar evolução real, curva ascendente do patrimonio atual até o valor futuro, com as contribuições, no ano de usufruto e curva descendente com as retiradas até a idade máxima.

In [148]:
class FinancialCalculator:
    

    TWO_CENTS = (Decimal("0.00"), ROUND_HALF_EVEN)
    FOR_PLACES = Decimal("0.0000")

    def __init__(
            self, 
            current_age, 
            goal_age, 
            goal_monthly_withdraw, 
            initial_patrimony, 
            monthly_contribution, 
            current_interest, 
            future_interest, 
            max_age
        ):
        self.current_age = current_age
        self.goal_age = goal_age
        self.goal_monthly_withdraw = Decimal(goal_monthly_withdraw)
        self.initial_patrimony = Decimal(initial_patrimony)
        self.monthly_contribution = Decimal(monthly_contribution)
        self.current_interest = Decimal().from_float(current_interest/100).quantize(self.FOR_PLACES, ROUND_HALF_EVEN)
        self.future_interest = Decimal().from_float(future_interest/100).quantize(self.FOR_PLACES, ROUND_HALF_EVEN)
        self.max_age = max_age

    @property
    def contribution_years(self):
        return self.goal_age - self.current_age

    @property
    def contribution_months(self):
        return self.contribution_years * 12

    @property
    def consumption_months(self):
        return (self.max_age - self.goal_age) * 12
    
    @property
    def monthly_yield_current(self):
        return self._convert_to_monthly_yield(self.current_interest)

    @property
    def monthly_yield_goal(self):
        return self._convert_to_monthly_yield(self.future_interest)

    @property
    def consumable_patrimony_value(self):
        return self.goal_monthly_withdraw * ((1 + self.monthly_yield_goal)**self.consumption_months - 1)/((1 + self.monthly_yield_goal)**self.consumption_months * self.monthly_yield_goal)

    @property
    def profitable_patrimony_value(self):
        return (self.goal_monthly_withdraw/self.monthly_yield_goal).quantize(*self.TWO_CENTS)
    
    def create_monthly_growth_history(self):
        last_month_value = self.initial_patrimony
        growth_history = [last_month_value]
        for i in range(0, self.contribution_months + 1):
            month_plus_interest = (last_month_value * (1 + self.monthly_yield_current)) + self.monthly_contribution
            month_plus_interest.quantize(*self.TWO_CENTS)
            growth_history.append(month_plus_interest)
            last_month_value = month_plus_interest
        return growth_history

    def create_anual_growth_history(self):
        last_year_value = self.initial_patrimony
        growth_history = [last_year_value]
        for i in range(0, self.contribution_years + 1):
            year_plus_interest = (last_year_value * (1 + self.current_interest)) + (self.monthly_contribution * 12)
            year_plus_interest.quantize(*self.TWO_CENTS)
            growth_history.append(year_plus_interest)
            last_month_value = year_plus_interest
        return growth_history


    def create_yearly_growth_table(self):
        pass

    def create_patrimony_consume_table(self):
        pass
    
    def create_yield_consume_table(self):
        pass

    @staticmethod
    def _convert_to_monthly_yield(anual_profitability):
        FIVE_PLACES = Decimal("0.000000")
        value = Decimal((1 + anual_profitability)**Decimal(1/12) - 1)
        return value.quantize(FIVE_PLACES, ROUND_HALF_EVEN)
    
    @staticmethod
    def _to_percent(value):
        return Decimal(value/100)

In [149]:
payload = {
            "current_age": 30,
            "goal_age": 60,
            "goal_monthly_withdraw": 2500,
            "initial_patrimony": 100000,
            "monthly_contribution": 100,
            "current_interest": 9,
            "future_interest":6,
            "max_age": 95
        }

In [150]:
data = FinancialCalculator(**payload)

InvalidOperation: [<class 'decimal.InvalidOperation'>]

In [ ]:
data.current_interest

In [ ]:
data.consumable_patrimony_value

In [151]:
data.profitable_patrimony_value

DivisionByZero: [<class 'decimal.DivisionByZero'>]

In [84]:
data.__dict__

{'current_age': 30,
 'goal_age': 60,
 'goal_monthly_withdraw': Decimal('2500'),
 'initial_patrimony': Decimal('100000'),
 'monthly_contribution': Decimal('100'),
 'current_interest': Decimal('0.0900'),
 'future_interest': Decimal('0.0600'),
 'max_age': 95}

In [96]:
growth_history = data.create_monthly_growth_history()
growth_history[:5]

TypeError: cannot unpack non-iterable decimal.Decimal object

In [12]:
growth_serie = pd.Series(growth_history)
growth_serie[:5]

0       100000
1    101080.70
2    102169.19
3    103265.52
4    104369.76
dtype: object

In [13]:
growth_serie.count()

361

In [14]:
data.__dict__

{'current_age': 30,
 'goal_age': 60,
 'goal_monthly_withdraw': Decimal('2500'),
 'initial_patrimony': Decimal('100000'),
 'monthly_contribution': Decimal('100'),
 'current_interest': Decimal('0.0900'),
 'future_interest': Decimal('0.0600'),
 'max_age': 95}

In [15]:
hoje = date.today()
hoje

datetime.date(2021, 1, 19)

In [16]:
goal_date = hoje + relativedelta(years=+(data.goal_age - data.current_age))
goal_date

datetime.date(2051, 1, 19)

In [17]:
max_date = hoje + relativedelta(years=(data.max_age - data.current_age))
max_date

datetime.date(2086, 1, 19)

In [18]:
# Lista de meses da data atual até a data máxima
months_list = pd.date_range(start=hoje, end=max_date, freq='M')
months_list

DatetimeIndex(['2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30',
               '2021-05-31', '2021-06-30', '2021-07-31', '2021-08-31',
               '2021-09-30', '2021-10-31',
               ...
               '2085-03-31', '2085-04-30', '2085-05-31', '2085-06-30',
               '2085-07-31', '2085-08-31', '2085-09-30', '2085-10-31',
               '2085-11-30', '2085-12-31'],
              dtype='datetime64[ns]', length=780, freq='M')

In [19]:
# Converte o DateTimeIndex para uma série
months_list = months_list.to_series().apply(lambda date:f"{date.month}/{str(date.year)[2::]}")

months_list

2021-01-31     1/21
2021-02-28     2/21
2021-03-31     3/21
2021-04-30     4/21
2021-05-31     5/21
              ...  
2085-08-31     8/85
2085-09-30     9/85
2085-10-31    10/85
2085-11-30    11/85
2085-12-31    12/85
Freq: M, Length: 780, dtype: object

## Função - Meses, números para nomes

In [20]:
def to_label(month):
    months = ['janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']
    month_info = month.split('/')
    return f"{months[int(month_info[0]) - 1]}/{month_info[1]}"

In [21]:
months_list = months_list.apply(to_label)
months_list

2021-01-31      janeiro/21
2021-02-28    fevereiro/21
2021-03-31        março/21
2021-04-30        abril/21
2021-05-31         maio/21
                  ...     
2085-08-31       agosto/85
2085-09-30     setembro/85
2085-10-31      outubro/85
2085-11-30     novembro/85
2085-12-31     dezembro/85
Freq: M, Length: 780, dtype: object

In [22]:
months_list.count()

780

In [23]:
month_series = pd.Series(months_list)

In [24]:
month_series

2021-01-31      janeiro/21
2021-02-28    fevereiro/21
2021-03-31        março/21
2021-04-30        abril/21
2021-05-31         maio/21
                  ...     
2085-08-31       agosto/85
2085-09-30     setembro/85
2085-10-31      outubro/85
2085-11-30     novembro/85
2085-12-31     dezembro/85
Freq: M, Length: 780, dtype: object

In [25]:
data.consumable_patrimony_value

Decimal('446753.9130176059371728841892')

In [26]:
def create_monthly_consume_history(consumable_patrimony_value, consume_months, monthly_withdraw, monthly_yield):
    TWO_CENTS = Decimal("0.00")
    last_month = consumable_patrimony_value
    consumption_history = []
    for month in range(0,consume_months - 1):
        if last_month == 0:
            consumption_history.append(0)
            continue
        avaliable_value = (last_month * (1 + monthly_yield)) - monthly_withdraw
        if avaliable_value > 0:
            consumption_history.append(avaliable_value.quantize(TWO_CENTS, ROUND_HALF_EVEN))
            last_month = avaliable_value
        else:
            consumption_history.append(0)
            last_month = 0
            
    return consumption_history

In [27]:
consumption_history = create_monthly_consume_history(growth_history[-1], data.consumption_months, data.goal_monthly_withdraw, data.monthly_yield_goal)
consumption_history[0:5]

[Decimal('1946266.75'),
 Decimal('1953241.18'),
 Decimal('1960249.55'),
 Decimal('1967292.05'),
 Decimal('1974368.83')]

In [28]:
consumption_series = pd.Series(consumption_history)
consumption_series

0       1946266.75
1       1953241.18
2       1960249.55
3       1967292.05
4       1974368.83
          ...     
414    11211265.01
415    11263341.45
416    11315671.40
417    11368256.08
418    11421096.75
Length: 419, dtype: object

In [29]:
consumption_series.get(key=418) > 0

True

In [30]:
consumption_series.count() + growth_serie.count()

780

In [31]:
client_history = growth_serie.append(consumption_series)

In [32]:
client_history.count()

780

In [33]:
client_history.tail()

414    11211265.01
415    11263341.45
416    11315671.40
417    11368256.08
418    11421096.75
dtype: object

In [34]:
df_schema = {
    'months': month_series,
    'history': client_history.reset_index(drop=True)
}

In [35]:
client_df = pd.DataFrame(df_schema)

In [38]:
client_df

,months,history
1970-01-01 00:00:00.000000000,NaN,NaN
1970-01-01 00:00:00.000000001,NaN,NaN
1970-01-01 00:00:00.000000002,NaN,NaN
1970-01-01 00:00:00.000000003,NaN,NaN
1970-01-01 00:00:00.000000004,NaN,NaN
...,...,...
2085-08-31 00:00:00.000000000,agosto/85,NaN
2085-09-30 00:00:00.000000000,setembro/85,NaN
2085-10-31 00:00:00.000000000,outubro/85,NaN
2085-11-30 00:00:00.000000000,novembro/85,NaN


In [39]:
data.__dict__


{'current_age': 30,
 'goal_age': 60,
 'goal_monthly_withdraw': Decimal('2500'),
 'initial_patrimony': Decimal('100000'),
 'monthly_contribution': Decimal('100'),
 'current_interest': Decimal('0.0900'),
 'future_interest': Decimal('0.0600'),
 'max_age': 95}

In [40]:
def get_present_value(future_value, interest_tax, time):
    return future_value/((1 + interest_tax)**time)

In [43]:
def valorfuturo(aporte, juros, periodo):
    return aporte*(((1 + juros)**periodo) - 1)/juros

In [47]:
data.consumable_patrimony_value

Decimal('446753.9130176059371728841892')